In [14]:
from dotenv import load_dotenv

load_dotenv()  # .env 파일을 환경 변수로 로드

True

In [15]:
"""
요약

Memory 를 Chain에 꽂는 방법
LLM Chain은 off-the-shelf 일반적인 목적을 가진 chain
off-the-shelf 보다는 직접 커스텀해서 chain 생성하는걸 선호함
"""
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [16]:
llm = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
)

template="""
    You are a helpful AI talking to a human

    {chat_history}
    Human:{question}
    You:
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True # chain의 프롬프트 로그 확인 가능
)

In [17]:
chain.predict(question="My name is Dooseop")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human

    
    Human:My name is Dooseop
    You:


> Finished chain.


'Hello Dooseop! How can I assist you today?'

In [18]:
chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human

    Human: My name is Dooseop
AI: Hello Dooseop! How can I assist you today?
    Human:I live in Seoul
    You:


> Finished chain.


"That's great to hear! How can I assist you with living in Seoul today?"

In [19]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human

    Human: My name is Dooseop
AI: Hello Dooseop! How can I assist you today?
Human: I live in Seoul
AI: That's great to hear! How can I assist you with living in Seoul today?
    Human:What is my name?
    You:


> Finished chain.


'Your name is Dooseop.'